In [ ]:
import os
import numpy as np
from pandas import read_csv
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from read_data import ReadClass

rc('text', usetex=True)
rc('font', family='serif')
fs = 16

figure_directory = "./figure"
if not os.path.isdir(figure_directory):
    os.makedirs(figure_directory)

finish_factor = 15


def derive_Thorpe_scale(experiment_name, index_finish):
    reader = ReadClass(experiment_name)
    reader.read_geometry(index_finish)
    Thorpe_PDF, Thorpe_axis = reader.read_Thorpe_displacement()
    Thorpe_array = []
    for it in range(np.size(reader.time)):
        Thorpe_scale = np.sqrt(
            np.sum(Thorpe_PDF[:, it] * Thorpe_axis[:]**2)
            / np.sum(Thorpe_PDF[:, it])
        );
        Thorpe_array.append(Thorpe_scale)
    return np.array(Thorpe_array)


def derive_Thorpe_scale_exclude_0(experiment_name, index_finish):
    reader = ReadClass(experiment_name)
    reader.read_geometry(index_finish)
    Thorpe_PDF, Thorpe_axis = reader.read_Thorpe_displacement()
    Thorpe_array = []
    for it in range(np.size(reader.time)):
        Thorpe_scale = np.sqrt(
            (np.sum(Thorpe_PDF[:reader.n3-1, it] * Thorpe_axis[:reader.n3-1]**2)
             + np.sum(Thorpe_PDF[reader.n3:, it] * Thorpe_axis[reader.n3:]**2))
            / (np.sum(Thorpe_PDF[:reader.n3-1, it]) + np.sum(Thorpe_PDF[reader.n3:, it]))
        );
        Thorpe_array.append(Thorpe_scale)
    return np.array(Thorpe_array)


def process_data(experiment_name, lambda_f):
    reader = ReadClass(experiment_name)
    num = round(reader.vortex_period / reader.output_interval) 
    b = np.ones(num) / num
    tau_growth = 1 / (2 * lambda_f * reader.f)
    index_finish = int(tau_growth * finish_factor / reader.output_interval)
    reader.read_geometry(index_finish)
    reader.read_budget(index_finish)
    DKE = reader.DKE / reader.output_interval
    DPE = reader.DPE / reader.output_interval
    WE_mean = np.convolve(reader.WE, b, mode='same')
    VE_mean = np.convolve(reader.VE, b, mode='same')
    PWE_mean = np.convolve(reader.PWE, b, mode='same')
    KE_mean = np.convolve(reader.KE, b, mode='same')
    PE_mean = np.convolve(reader.PE, b, mode='same')
    DKE_mean = np.convolve(DKE, b, mode='same')
    DPE_mean = np.convolve(DPE, b, mode='same')
    WE_max = np.max(WE_mean)
    VE_max = np.max(VE_mean)
    PWE_max = np.max(PWE_mean)
    KE_max = np.max(KE_mean)
    PE_max = np.max(PE_mean)
    DKE_max = np.max(DKE_mean)
    DPE_max = np.max(DPE_mean)
    DE_max = np.max(DPE_mean + DPE_mean)
    N = reader.buoyancy_frequency
    e = reader.e
    Ro = reader.Ro
    Thorpe = derive_Thorpe_scale(experiment_name, index_finish)
    Thorpe_mean = np.convolve(Thorpe, b, mode='same')
    Thorpe_scale = np.nanmax(Thorpe_mean)
    return (KE_max, PE_max, DKE_max, DPE_max, DE_max,
            WE_max, VE_max, PWE_max, Thorpe_scale, N, e, Ro)


def cumulative_production(experiment_name, lambda_f):
    reader = ReadClass(experiment_name)
    tau_growth = 1 / (2 * lambda_f * reader.f)
    index_finish = int(tau_growth * finish_factor / reader.output_interval)
    reader.read_budget(index_finish)
    PKE_cum = np.cumsum(reader.PKE)[-1]
    PW1_cum = np.cumsum(reader.PW1)[-1]
    PW2_cum = np.cumsum(reader.PW2)[-1]
    PVE_cum = np.cumsum(reader.PVE)[-1]
    return  PKE_cum, PW1_cum, PW2_cum, PVE_cum


csv_input = read_csv(
    filepath_or_buffer="../control/parameters_scatter.csv",
    encoding="ms932", sep=",", index_col="name"
)

Thorpe_array = []
N_array = []
NF_array = []
e_array = []
Ro_array = []
lambda_N_array = []
lambda_f_array = []
WE_array = []
VE_array = []
PWE_array = []
KE_array = []
PE_array = []
DKE_array = []
DPE_array = []
DE_array = []
PKE_array = []
PW1_array = []
PW2_array = []
PVE_array = []

for experiment_name in csv_input.index.values:
    print(experiment_name)
    lambda_N = csv_input.at[experiment_name, 'lamN']
    lambda_N_array.append(lambda_N)
    lambda_f = csv_input.at[experiment_name, 'lamF']
    lambda_f_array.append(lambda_f)
    NF_array.append(csv_input.at[experiment_name, 'NF'])
    (KE_max, PE_max, DKE_max, DPE_max, DE_max, WE_max, VE_max, PWE_max,
     Thorpe_scale, N, e, Ro) = process_data(
         experiment_name, lambda_f)
    PKE_cum, PW1_cum, PW2_cum, PVE_cum = cumulative_production(
        experiment_name, lambda_f)
    N_array.append(N)
    e_array.append(e)
    Ro_array.append(Ro)
    WE_array.append(WE_max)
    VE_array.append(VE_max)
    PWE_array.append(PWE_max)
    KE_array.append(KE_max)
    PE_array.append(PE_max)
    DKE_array.append(DKE_max)
    DPE_array.append(DPE_max)
    DE_array.append(DE_max)
    PKE_array.append(PKE_cum)
    PW1_array.append(PW1_cum)
    PW2_array.append(PW2_cum)
    PVE_array.append(PVE_cum)
    Thorpe_array.append(Thorpe_scale)

N_na = np.array(N_array)
NF_na = np.array(NF_array)
e_na = np.array(e_array)
Ro_na = np.array(Ro_array)
lambda_N_na = np.array(lambda_N_array)
lambda_f_na = np.array(lambda_f_array)
WE_na = np.array(WE_array)
VE_na = np.array(VE_array)
PWE_na = np.array(PWE_array)
KE_na = np.array(KE_array)
PE_na = np.array(PE_array)
DKE_na = np.array(DKE_array)
DPE_na = np.array(DPE_array)
DE_na = np.array(DE_array)
PKE_na = np.array(PKE_array)
PW1_na = np.array(PW1_array)
PW2_na = np.array(PW2_array)
PVE_na = np.array(PVE_array)
Thorpe_na = np.array(Thorpe_array)

mp = 56

In [ ]:
fig = plt.figure(figsize=[6, 12])
fig.subplots_adjust(left=0.15, bottom=0.12, right=0.96,
                    top=0.94, wspace=0.2, hspace=0.25)

ax1 = fig.add_subplot(311)
sc = ax1.scatter(
    lambda_N_na[:mp], (DKE_na[:mp] / N_na[:mp]**3)**0.5 * (2 * np.pi / 200),
    c=NF_na[:mp], vmin=0, vmax=10, zorder=20
);
cbar = plt.colorbar(sc);
cbar.ax.set_title(r'$N/f$', fontsize=fs)
ax1.grid(zorder=10)
ax1.set_xlabel(r'$\lambda/N$', fontsize=fs)
ax1.set_ylabel(r'$\eta_O m_p$', fontsize=fs)
ax1.set_title(r'(a)', loc='left', fontsize=fs)

grid_x_ticks_mimor = np.arange(0, 0.035, 0.001)
grid_y_ticks_mimor = np.arange(0, 0.16, 0.01)
grid_y_ticks_major = np.arange(0, 0.20, 0.05)

ax1.set_xticks(grid_x_ticks_mimor , minor=True)
ax1.set_yticks(grid_y_ticks_mimor , minor=True)
ax1.set_yticks(grid_y_ticks_major , major=True)
ax1.set_xlim([0, 0.03]);
ax1.set_ylim([0, 0.16]);

x1_na = np.linspace(0.0012, 0.006)
x2_na = np.linspace(0.009, 0.025)
axins1 = inset_axes(ax1, width=1.5, height=1.5, loc=4, borderpad=0.5)
sc2 = axins1.scatter(
    lambda_N_na[:mp], (DKE_na[:mp] / N_na[:mp]**3)**0.5 * (2 * np.pi / 200),
    c=NF_na[:mp], s=3, vmin=0, vmax=10, zorder=20
);
axins1.plot(x1_na, 12*x1_na, 'k', linewidth=1, zorder=15)
axins1.plot(x2_na, 1.1*x2_na**0.5, 'k', linewidth=1, zorder=15)
axins1.set_xlim([0.0005, 0.05]);
axins1.set_ylim([0.005, 0.5]);
axins1.set_xscale('log')
axins1.set_yscale('log')
axins1.set_xticklabels([])
axins1.set_yticklabels([])
axins1.grid(zorder=10)

ax2 = fig.add_subplot(312)

x_na = np.linspace(0, 2)
y_na = x_na
sc = ax2.scatter(
    lambda_f_na[:mp] * 1.e-4 * 2 * 86400,
    DKE_na[:mp] / PWE_na[:mp] * 86400,
    c=NF_na[:mp], vmin=0, vmax=10, zorder=20
);

ax2.plot(x_na, y_na, 'k', zorder=15)
cbar = plt.colorbar(sc);
cbar.ax.set_title(r'$N/f$', fontsize=fs)
ax2.grid(zorder=10)
ax2.set_xlabel(r'$\tau_g^{-1}$ [/day]', fontsize=fs)
ax2.set_ylabel(r'$\tau_d^{-1}$ [/day]', fontsize=fs)
ax2.set_title(r'(c)', loc='left', fontsize=fs)

grid_x_ticks_mimor = np.arange(0, 1.3, 0.1)
grid_x_ticks_major = np.arange(0, 1.4, 0.2)
grid_y_ticks_mimor = np.arange(0, 1.3, 0.1)
grid_y_ticks_major = np.arange(0, 1.4, 0.2)

ax2.set_xticks(grid_x_ticks_mimor , minor=True)
ax2.set_xticks(grid_x_ticks_major , major=True)
ax2.set_yticks(grid_y_ticks_mimor , minor=True)
ax2.set_yticks(grid_y_ticks_major , major=True)
ax2.set_xlim([0, 1.3]);
ax2.set_ylim([0, 1.3]);

axins2 = inset_axes(ax2, width=1.5, height=1.5, loc=2, borderpad=.5)
sc2 = axins2.scatter(
    lambda_f_na[:mp] * 1.e-4 * 2 * 86400,
    DKE_na[:mp] / PWE_na[:mp] * 86400,
    c=NF_na[:mp], s=3, vmin=0, vmax=10, zorder=20
);
axins2.plot(x_na, y_na, 'k', linewidth=1, zorder=15)
axins2.set_xlim([0.05, 2]);
axins2.set_ylim([0.05, 2]);
axins2.set_xscale('log')
axins2.set_yscale('log')
axins2.set_xticklabels([])
axins2.set_yticklabels([])
axins2.grid(zorder=10)

ax3 = fig.add_subplot(313)

x_na = np.linspace(5.e-4, 3.e-2)
sc = ax3.scatter(
    lambda_N_na[:mp],
    (PWE_na[:mp] * 2 / N_na[:mp]**2 * (2 * np.pi / 200)**2)**0.5,
    c=NF_na[:mp], vmin=0, vmax=10, zorder=20
);

cbar = plt.colorbar(sc);
cbar.ax.set_title(r'$N/f$', fontsize=fs)
ax3.grid(zorder=10)
ax3.set_xlabel(r'$\lambda/N$', fontsize=fs)
ax3.set_ylabel(r'$Fr_p$', fontsize=fs)
ax3.set_title(r'(b)', loc='left', fontsize=fs)

grid_x_ticks_mimor = np.arange(0, 0.035, 0.001)
grid_y_ticks_mimor = np.arange(0, 1.1, 0.1)
grid_y_ticks_major = np.arange(0, 1.2, 0.2)

ax3.set_xticks(grid_x_ticks_mimor , minor=True)
ax3.set_yticks(grid_y_ticks_mimor , minor=True)
ax3.set_yticks(grid_y_ticks_major , major=True)
ax3.set_xlim([0, 0.03]);
ax3.set_ylim([0, 1.0]);

x1_na = np.linspace(0.0012, 0.006)
x2_na = np.linspace(0.01, 0.03)
axins3 = inset_axes(ax3, width=1.5, height=1.5, loc=4, borderpad=0.5)
sc2 = axins3.scatter(
    lambda_N_na[:mp],
    (PWE_na[:mp] * 2 / N_na[:mp]**2 * (2 * np.pi / 200)**2)**0.5,
    c=NF_na[:mp], s=3, vmin=0, vmax=10, zorder=20
);
axins3.plot(x1_na, 12*x1_na**0.5, 'k', linewidth=1, zorder=15)

axins3.set_xlim([0.0005, 0.05]);
axins3.set_ylim([0.05, 5]);
axins3.set_xscale('log')
axins3.set_yscale('log')
axins3.set_xticklabels([])
axins3.set_yticklabels([])
axins3.grid(zorder=10)

figure_name = "scatter_lambda"
fig.savefig(figure_directory + '/' + figure_name + '.eps');
fig.savefig(figure_directory + '/' + figure_name + '.png');

In [ ]:
fig = plt.figure(figsize=[6, 4])
fig.subplots_adjust(left=0.15, bottom=0.12, right=0.96,
                    top=0.94, wspace=0.2, hspace=0.25)
ax1 = fig.add_subplot(111)

x_na = np.linspace(0, 5.0)

sc = ax1.scatter(
    (DKE_na[:mp] / N_na[:mp]**3)**0.5,
    Thorpe_na[:mp],
    c=lambda_N_na[:mp], vmin=0, vmax=0.03, zorder=20, cmap=cm.plasma #cm.jet
);
ax1.plot(x_na, 2.5 * x_na, 'k')
ax1.plot(x_na, 5 * x_na, 'k')
cbar = plt.colorbar(sc);
cbar.ax.set_title(r'$\lambda/N$', fontsize=fs)
ax1.grid(zorder=10)
ax1.set_xlabel(r'$\eta_O$ [m]', fontsize=fs)
ax1.set_ylabel(r'$\eta_T$ [m]', fontsize=fs)

grid_x_ticks_mimor = np.arange(0, 5.1, 0.1)
grid_x_ticks_major = np.arange(0, 5.5, 0.5)
grid_y_ticks_mimor = np.arange(0, 16, 1)
grid_y_ticks_major = np.arange(0, 16, 2)

ax1.set_xticks(grid_x_ticks_mimor , minor=True)
ax1.set_xticks(grid_x_ticks_major , major=True)
ax1.set_yticks(grid_y_ticks_mimor , minor=True)
ax1.set_yticks(grid_y_ticks_major , major=True)
ax1.set_xlim([0, 5]);
ax1.set_ylim([0, 15]);

axins = inset_axes(ax1, width=1.3, height=1.1, loc=4, borderpad=2)
sc = axins.scatter(
    lambda_N_na[:mp],
    Thorpe_na[:mp] / (DKE_na[:mp] / N_na[:mp]**3)**0.5,
    marker='o', s=10, c="cyan", edgecolor='blue', alpha=0.5, linewidths=0.5,
    zorder=20,
);

axins.grid(zorder=10)
axins.plot(x_na, y_na, 'k', zorder=15)
axins.set_xticks([0, 0.01, 0.02, 0.03]);
axins.set_xlim([0, 0.03]);
axins.set_ylim([0, 5]);

grid_x_ticks_mimor = np.arange(0, 0.035, 0.001)
grid_y_ticks_mimor = np.arange(0, 11, 1)
grid_y_ticks_major = np.arange(0, 12, 2)

axins.set_xticks(grid_x_ticks_mimor , minor=True)
axins.set_yticks(grid_y_ticks_mimor , minor=True)
axins.set_yticks(grid_y_ticks_major , major=True)
axins.set_xlim([0, 0.03]);
axins.set_ylim([0, 10]);

figure_name = "Ozmidov_Thorpe"
fig.savefig(figure_directory + '/' + figure_name + '.eps');
fig.savefig(figure_directory + '/' + figure_name + '.png');

In [ ]:
fig = plt.figure(figsize=[6, 12])
fig.subplots_adjust(left=0.15, bottom=0.08, right=0.79,
                    top=0.96, wspace=0.2, hspace=0.25)
ax1 = fig.add_subplot(311)

sc = ax1.scatter(
    WE_na[:mp] / (200**2 * N_na[:mp]**2),
    VE_na[:mp] / (200**2 * N_na[:mp]**2),
    c=lambda_N_na[:mp], vmin=0, vmax=0.03, zorder=20, cmap=cm.plasma #cm.jet
);

cax = fig.add_axes((0.83, 0.707, 0.028, 0.255))
cbar = fig.colorbar(sc, cax=cax)
cbar.ax.set_title(r'$\lambda/N$', fontsize=fs)
ax1.grid(zorder=10)
ax1.set_xlabel(r'$E_w / (L_z^2 N^2)$', fontsize=fs)
ax1.set_ylabel(r'$E_v / (L_z^2 N^2)$', fontsize=fs)

grid_x_ticks_minor = np.arange(0, 0.021, 0.001)
grid_x_ticks_major = np.arange(0, 0.025, 0.005)
grid_y_ticks_minor = np.arange(0, 0.0031, 0.0001)
grid_y_ticks_major = np.arange(0, 0.004, 0.001)

ax1.set_xticks(grid_x_ticks_minor , minor=True)
ax1.set_xticks(grid_x_ticks_major , major=True)
ax1.set_yticks(grid_y_ticks_minor , minor=True)
ax1.set_yticks(grid_y_ticks_major , major=True)
ax1.set_xlim([0, 0.02]);
ax1.set_ylim([0, 0.003]);
ax1.set_title('(a)', loc='left', fontsize=fs);

axins1 = inset_axes(ax1, width=1.3, height=1.3,
                    bbox_transform=ax1.transAxes,
                    loc=2, bbox_to_anchor=(.08, .48, .6, .5))
sc = axins1.scatter(
    lambda_N_na[:mp],
    VE_na[:mp] / WE_na[:mp],
    marker='o', s=10, c="cyan", edgecolor='blue', alpha=0.5, linewidths=0.5,
    zorder=20,
);

axins1.grid(zorder=10)

grid_x_ticks_minor = np.arange(0, 0.031, 0.001)
grid_x_ticks_major = np.arange(0, 0.04, 0.01)
grid_y_ticks_minor = np.arange(0, 0.31, 0.01)
grid_y_ticks_major = np.arange(0, 0.4, 0.1)

axins1.set_xticks(grid_x_ticks_minor , minor=True)
axins1.set_xticks(grid_x_ticks_major , major=True)
axins1.set_yticks(grid_y_ticks_minor , minor=True)
axins1.set_yticks(grid_y_ticks_major , major=True)
axins1.set_xlim([0, 0.03]);
axins1.set_ylim([0, 0.3]);

ax2 = fig.add_subplot(312)

sc1 = ax2.scatter(
    lambda_N_na[:mp],
    PW1_na[:mp] / PKE_na[:mp],
    zorder=20,
    marker='v', c=['r'], label=r'$\mathcal{P}^c_w / \mathcal{P}^c$'
);
sc2 = ax2.scatter(
    lambda_N_na[:mp],
    PW2_na[:mp] / PKE_na[:mp],
    zorder=20,
    marker='s', c=['b'], label=r'$\mathcal{P}^c_{wv} / \mathcal{P}^c$'
);
sc3 = ax2.scatter(
    lambda_N_na[:mp],
    PVE_na[:mp] / PKE_na[:mp],
    zorder=20,
    marker='x', c=['c'], label=r'$\mathcal{P}^c_v / \mathcal{P}^c$'
);
ax2.legend(bbox_to_anchor=(0.99, 1.0), loc='upper left',
           frameon=False,
           borderaxespad=0.,
           fontsize=14)
ax2.hlines([0, 1], 0, 0.03, color='k', zorder=5)
ax2.grid(zorder=10)
ax2.set_xlim([0, 0.03]);
ax2.set_xlabel(r'$\lambda / N$', fontsize=fs)
ax2.set_ylabel(r'$\mathcal{P}^c_{\ast} / \mathcal{P}^c$', fontsize=fs)
ax2.set_title('(b)', loc='left', fontsize=fs);

grid_x_ticks_mimor = np.arange(0, 0.035, 0.001)
grid_y_ticks_minor = np.arange(-0.2, 1.3, 0.1)
grid_y_ticks_major = np.arange(-0.2, 1.4, 0.2)

ax2.set_xticks(grid_x_ticks_mimor , minor=True)
ax2.set_yticks(grid_y_ticks_minor , minor=True)
ax2.set_yticks(grid_y_ticks_major , major=True)
ax2.set_xlim([0, 0.03]);
ax2.set_ylim([-0.2, 1.2]);

ax3 = fig.add_subplot(313)

index_strong = lambda_N_na[:mp] > 0.008
index_weak = lambda_N_na[:mp] < 0.008

sc = ax3.scatter(
    e_na[index_weak],
    PW2_na[index_weak] / PKE_na[index_weak],
    c=Ro_na[index_weak], vmin=0.9, vmax=1, cmap=cm.jet,
    marker='+',
    zorder=20,
);

sc = ax3.scatter(
    e_na[index_strong],
    PW2_na[index_strong] / PKE_na[index_strong],
    c=Ro_na[index_strong], vmin=0.9, vmax=1, cmap=cm.jet,
    zorder=20,
);
scl1 = ax3.scatter([],[],c='k', marker='+',
                  label=r'$\lambda / N < 0.008$')
scl2 = ax3.scatter([],[],c='k',
                  label=r'$\lambda / N > 0.008$')
ax3.legend(fontsize=14, loc=3)
ax3.hlines([0], 0, 1, color='k', zorder=5)
ax3.grid(zorder=10)

grid_x_ticks_minor = np.arange(0, 1.1, 0.1)
grid_x_ticks_major = np.arange(0, 1.1, 0.1)
grid_y_ticks_minor = np.arange(-0.2, 0.41, 0.01)
grid_y_ticks_major = np.arange(-0.2, 0.5, 0.1)

ax3.set_xticks(grid_x_ticks_minor, minor=True)
ax3.set_xticks(grid_x_ticks_major, major=True)
ax3.set_yticks(grid_y_ticks_minor, minor=True)
ax3.set_yticks(grid_y_ticks_major, major=True)

ax3.set_xlim([0, 1]);
ax3.set_ylim([-0.2, 0.4]);
ax3.set_xlabel(r'$e$', fontsize=fs)
ax3.set_ylabel(r'$\mathcal{P}^c_{wv} / \mathcal{P}^c$', fontsize=fs)

cax = fig.add_axes((0.83, 0.077, 0.028, 0.255))
cbar = fig.colorbar(sc, cax=cax, extend='min')
cbar.ax.set_title(r'$Ro$', fontsize=fs)

print(np.min(PKE_na), np.max(PKE_na), np.min(PW2_na), np.max(PW2_na))
print(np.argmin(PW2_na))

figure_name = "wave_vortex"
fig.savefig(figure_directory + '/' + figure_name + '.eps');
fig.savefig(figure_directory + '/' + figure_name + '.png');